In [1]:
%cd ../..

E:\システムトレード入門\trade_system_git_workspace


In [2]:
from tqdm.notebook import tqdm

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytz import timezone
import datetime
from pathlib import Path
import pandas as pd
import numpy as np
import pfrl

In [4]:
from scipy.special import softmax

In [5]:
import py_restart

In [6]:
from get_stock_price import StockDatabase

In [7]:
from portfolio.trade_transformer import PortfolioTransformer, PortfolioRestrictorIdentity, FeeCalculatorFree
from portfolio.price_supply import StockDBPriceSupplier

In [8]:
from portfolio.rl_base.envs import TradeEnv, TickerSampler, DatetimeSampler, SamplerManager, PortfolioVectorSampler, ConstSamper
from portfolio.rl_base.basis_func import ComposeFunction, PriceNormalizeConst, MeanCostPriceNormalizeConst, State2Feature

In [9]:
from portfolio.rl_base.make_env import make_env

In [10]:
from visualization import visualize_portfolio_rl_bokeh

## 並列環境の作成

In [11]:
one_trade_env = make_env()

In [12]:
portfolio_state, _,_,_ = one_trade_env.reset()
#print(portfolio_state)

In [13]:
class VectorEnv():
    def __init__(self,remote_envs):
        self._remote_envs = remote_envs 

    @property
    def remote_envs(self):
        return self._remote_envs

    def reset(self, *args, **kwargs):
        out_list = [remote_env.reset(*args, **kwargs) for remote_env in self._remote_envs]
        # それぞれの返り値の長さが複数かどうか
        if len(out_list) > 0:
            if isinstance(out_list[0], tuple):  # 一つの返り値が複数の場合
                return zip(*out_list)  # それぞれの出力のリストに変換(unzip)
            else:
                return out_list  # そのまま出力
        return None

    def step(self, actions, *args, **kwargs):
        assert len(self._remote_envs) == len(actions)
        out_list = [remote_env.step(action, *args, **kwargs) for remote_env, action in zip(self.remote_envs, actions)]
        if len(out_list) > 0:
            if isinstance(out_list[0], tuple):  # 一つの返り値が複数の場合
                return zip(*out_list)  # それぞれの出力のリストに変換(unzip)
            else:
                return out_list  # そのまま出力
        return None

In [14]:
env_number = 1
env_list = [make_env() for _ in range(env_number)]
batch_trade_env = VectorEnv(env_list)

In [15]:
states, rewards, dones, infos = batch_trade_env.reset()

In [16]:
#print(states[0])

## 前処理の設定 

In [17]:
def make_transform():
    return ComposeFunction({"price_normalizer":PriceNormalizeConst(None),
                                   "mca_normalizer":MeanCostPriceNormalizeConst(None),
                                   "state2feature":State2Feature()
                                  })

In [18]:
one_state_transform = make_transform()

In [19]:
class VectorTransform():
    def __init__(self, transform_list):
        self.transforms = transform_list
        
    def __call__(self, states):
        return [one_transform(state) for one_transform, state in zip(self.transforms, states)]

In [20]:
transform_list = [make_transform() for _ in range(env_number)]
batch_state_transform = VectorTransform(transform_list)

##  モデルの定義 

今回は決定論的な方策を利用する

In [21]:
class DirichletHead(nn.Module):
    def __init__(self):
        super(DirichletHead, self).__init__()
        
    def forward(self, alpha):
        return torch.distributions.Dirichlet(alpha)

In [22]:
class Policy(nn.Module):
    def __init__(self, in_channels=3, out_number=20):
        super(Policy, self).__init__()
        self.out_number = out_number
        
        self.conv1 = nn.Conv2d(in_channels, 12, kernel_size=5, padding=2, stride=2)
        self.bn1 = nn.BatchNorm2d(12)
        self.conv2 = nn.Conv2d(12, self.out_number, kernel_size=5, padding=2, stride=(2,3))
        self.bn2 = nn.BatchNorm2d(self.out_number)
        self.conv3 = nn.Conv2d(self.out_number, self.out_number, kernel_size=(4,3), padding=(2,1), stride=(2,3))
        #self.bn3 = nn.BatchNorm2d(self.out_number)

        #self.avgpool = nn.AvgPool2d(kernel_size=3)
        self.maxpool = nn.MaxPool2d(kernel_size=3)
        self.head = DirichletHead()
        
        
    def forward(self, x):
        #from IPython.core.debugger import Pdb; Pdb().set_trace()
        if x.shape[1:]!=torch.Size([3, 20, 50]):
            print(x)
            raise Exception("invalid shape policy")
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        #x = self.bn3(self.conv3(x))
        x = self.conv3(x)
        
        #x = self.avgpool(x)
        x = self.maxpool(x)
        x = torch.reshape(x, (-1, self.out_number))
        
        #x = F.softmax(x, dim=-1)
        x = F.relu(x)
        x = torch.clamp(x, 1.e-5)
        out = self.head(x)
        return out

In [23]:
#random_x = torch.randn((30, 3, 20, 50))
random_x = 100*torch.ones((1,3,20,50))
policy = Policy(3, 20)

In [24]:
out = policy(random_x)
print(out.batch_shape, out.event_shape)
print(out.rsample(torch.Size([])).shape)

torch.Size([1]) torch.Size([20])
torch.Size([1, 20])


In [25]:
class QFunc(nn.Module):
    def __init__(self, in_channels=3, action_size=20):
        super(QFunc, self).__init__()        
        self.conv1 = nn.Conv2d(in_channels, 12, kernel_size=5, padding=2, stride=2)
        self.bn1 = nn.BatchNorm2d(12)
        self.conv2 = nn.Conv2d(12, 24, kernel_size=5, padding=2, stride=(2,3))
        self.bn2 = nn.BatchNorm2d(24)
        self.conv3 = nn.Conv2d(24, 64, kernel_size=(4,3), padding=(2,1), stride=(2,3))
        self.bn3 = nn.BatchNorm2d(64)
        self.avgpool = nn.AvgPool2d(kernel_size=3)

        self.fc1 = nn.Linear(action_size, 64)
        self.fcbn1 = nn.BatchNorm1d(64)

        self.concat = pfrl.nn.ConcatObsAndAction()
        self.fc2 = nn.Linear(64*2, 256)
        self.fcbn2 = nn.BatchNorm1d(256)
        self.head = nn.Linear(256, 1)

    def forward(self, obs_and_action):
        #from IPython.core.debugger import Pdb; Pdb().set_trace()
        obs, action = obs_and_action
        x = F.relu(self.bn1(self.conv1(obs)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.avgpool(x)
        x = torch.reshape(x, (-1, 64))

        y = F.relu(self.fcbn1(self.fc1(action)))

        x = self.concat((x, y))
        x = F.relu(self.fcbn2(self.fc2(x)))
        out = self.head(x)
        return out

In [26]:
random_obs = torch.randn(30, 3, 20, 50)
random_action = torch.randn(30, 20)

In [27]:
qfunc = QFunc(3, 20)
out = qfunc((random_obs, random_action))

In [28]:
out.shape

torch.Size([30, 1])

## エージェントの作成 

In [29]:
policy = Policy(in_channels=3, out_number=20)
q_func1 = QFunc(in_channels=3, action_size=20)
q_func2 = QFunc(in_channels=3, action_size=20)

opt_p = torch.optim.Adam(policy.parameters())
opt_q1 = torch.optim.Adam(q_func1.parameters())
opt_q2 = torch.optim.Adam(q_func2.parameters())

rbuf = pfrl.replay_buffers.ReplayBuffer(1.e2)

def burnin_action_func():
    """Select random actions until model is updated one or more times."""
    random_x = np.random.uniform(np.array([0]*20), np.array([1]*20))
    out = softmax(random_x).astype(np.float32)
    return out
        
gpu = -1

gamma = 0.99 

replay_start_size =100

#batch_size = 256
batch_size = 16  # 環境の数と異なることに注意
    
phi = lambda x: x.astype(np.float32, copy=False)


sac_agent = pfrl.agents.SoftActorCritic(
    policy,
    q_func1,
    q_func2,
    opt_p,
    opt_q1,
    opt_q2,
    rbuf,
    phi=phi,
    gamma=gamma,
    replay_start_size=replay_start_size,
    gpu=gpu,
    minibatch_size=batch_size,
    burnin_action_func=burnin_action_func,
    entropy_target=-20,
    temperature_optimizer_lr=3e-4,
    act_deterministically=False
)

## 学習のための関数 

In [30]:
def episode(env,
            agent,
            state_transform,
            return_state_reward=True,
            field_list=["now_price_array", "portfoilo_vector", "mean_cost_price_array", "all_assets", "datetime"],
            seed=None,
            print_span=None,
            is_observe=True):
    
    state_list = []
    reward_list = []
    
    portfolio_state,reward,_,_ = env.reset(seed)
    
    #state_transformの設定
    state_transform.price_normalizer.const_array = portfolio_state.now_price_array
    state_transform.mca_normalizer.const_array = portfolio_state.now_price_array
    
    
    state_list.append(portfolio_state.partial(*field_list))
    reward_list.append(reward)
    

    R = 0
    t = 1
    
    obs = state_transform(portfolio_state)
    
    if print_span is not None:
        print("initial:, all_assets:{}".format(portfolio_state.all_assets))

    while True:
        action = agent.act(obs)
        portfolio_state, reward, done, info = env.step(action)

        state_list.append(portfolio_state.partial(*field_list))
        reward_list.append(reward)

        R += reward
        t += 1
        reset = False
        
        # state前処理
        obs = state_transform(portfolio_state)

        if is_observe:  # 観測(学習)する場合
            agent.observe(obs, reward, done, reset)

        if done:
            break
        if print_span is not None:
            if t%print_span==0:
                print("\tt={}:, all_assets:{}".format(t,portfolio_state.all_assets))
    
    if print_span is not None:
        print("finished(t={}):, all_assets:{}".format(t, portfolio_state.all_assets))
    
    out_dict = {}
    out_dict["R"] = R 
    
    if return_state_reward:
        out_dict["state_list"] = state_list
        out_dict["reward_list"] = reward_list
            
    return out_dict

In [31]:
def batch_episode(batch_env,
            agent,
            batch_state_transform,
            seed=None,
            print_span=None,
            is_observe=True):
    #from IPython.core.debugger import Pdb; Pdb().set_trace()
        
    portfolio_states, rewards,_,_ = batch_env.reset(seed)
    
    #state_transformの設定
    for one_portfolio_state, one_state_transform in zip(portfolio_states, batch_state_transform.transforms):
        one_state_transform.price_normalizer.const_array = one_portfolio_state.now_price_array
        one_state_transform.mca_normalizer.const_array = one_portfolio_state.now_price_array
    
    R = np.zeros(len(portfolio_states))
    t = 1
    
    obss = batch_state_transform(portfolio_states)
    
    if print_span is not None:
        print("initial:, all_assets:{}".format(portfolio_states[0].all_assets))

    while True:
        actions = agent.batch_act(obss)
        portfolio_states, rewards, dones, _ = batch_env.step(actions)

        R += rewards
        t += 1
        resets = [False] * len(portfolio_states)
        
        # state前処理
        obss = batch_state_transform(portfolio_states)

        if is_observe:  # 観測(学習)する場合
            agent.batch_observe(obss, rewards, dones, resets)

        if any(dones):
            break
        if print_span is not None:
            if t%print_span==0:
                print("\tt={}:, all_assets:{}".format(t,portfolio_states[0].all_assets))
    
    if print_span is not None:
        print("finished(t={}):, all_assets:{}".format(t, portfolio_states[0].all_assets))
        
    return {"R":R}

##  エージェント名

In [32]:
now_datetime = datetime.datetime.now()
now_str = now_datetime.strftime("%Y_%m_%d__%H_%M_%S")
agent_name = "sac_"+now_str
print(agent_name)

sac_2021_05_17__11_17_49


## 一時保存用のオブジェクト

In [33]:
temp_save_dict = {"agent_name":agent_name}

## 一時保存用の設定 

In [34]:
temp_filepath = Path("portfolio/rl_base/training_temp_sac.tmp")

object_temp_filepath = Path("portfolio/rl_base/temp_save_dict_sac.pickle")

save_funcs = [sac_agent.save]
load_funcs = [sac_agent.load]
func_paths = [Path("portfolio/rl_base/temp_agent_sac")]

## オブジェクトの登録とロード

In [35]:
counter = py_restart.enable_counter(temp_filepath, each_save=True, save_span=100, use_tempfile=True)

temp_save_dict = counter.save_load_object(temp_save_dict, object_temp_filepath)

counter.save_load_funcs(save_funcs=save_funcs,
                        load_funcs=load_funcs,
                        func_paths=func_paths
                        )

## 画像を保存するディレクトリ 

In [36]:
save_fig_dir_path = Path("portfolio/rl_base/trading_process_figures") / Path(agent_name)
if not save_fig_dir_path.exists():
    save_fig_dir_path.mkdir()

## 学習のパラメータ― 

In [37]:
n_episodes = 1000

## 学習のイテレーション 

In [ ]:
with counter:
    for i in counter(tqdm(range(1, n_episodes + 1))):
        out_dict = batch_episode(batch_trade_env,
                      sac_agent,
                      batch_state_transform,
                     )        

        if i%50 == 0:
            print("episode:{}, return:{}".format(i, out_dict["R"]))
        if i%100 == 0:
            print("statistics:", sac_agent.get_statistics())

        if i%50 == 0:
            with sac_agent.eval_mode():
                out_dict = episode(one_trade_env, 
                                   sac_agent,
                                   one_state_transform,
                                   return_state_reward=True,
                                   field_list=["names", "now_price_array", "portfolio_vector", "mean_cost_price_array", "all_assets", "datetime"],
                                   is_observe=False
                                   )
                
                save_fig_path = save_fig_dir_path / Path("trading_process_i_{}.png".format(i))
                visualize_portfolio_rl_bokeh(out_dict["state_list"], out_dict["reward_list"], save_path=save_fig_path, is_show=False, is_save=True, is_jupyter=True)
            
print("Finshed")

## 学習結果の可視化

In [ ]:
with sac_agent.eval_mode():
    out_dict = episode(trade_env, 
                       ddpg_agent,
                       state_transform,
                       return_state_reward=True,
                       field_list=["names", "now_price_array", "portfolio_vector", "mean_cost_price_array", "all_assets", "datetime"],
                       is_observe=False
                       )
    
visualize_portfolio_rl_bokeh(out_dict["state_list"], out_dict["reward_list"], is_show=False, is_jupyter=True)

In [ ]:
save_agent_path = Path("portfolio/rl_base/saved_agents") / Path(agent_name)
sac_agent.save(save_agent_path)